In [84]:
import numpy as np
from scipy.stats import norm

In [85]:
S     = 1.1325  # Spot rate
r_d   = 0.0433  # Domestic interest rate
r_f   = 0.02167 # Foreign interest rate
T     = 1.0     # Maturity in years
sigma = 0.0895  # Volatility

K_1 = 0.9 * S   # Strike of first option
K_2 = 1.10 * S  # Strike of second option

In [86]:
def garman_kohlhagen(S: float, K: float, r_d: float, r_f: float, T: float, sigma: float) -> tuple[float, float, float, float]:
    """Derivative pricing according to Garman-Kohlhagen model

    Args:
        S (float): Spot rate
        K (float): Strike
        r_d (float): domestic risk-free interest rate
        r_f (float): foreign risk-free interest rate
        T (float): Maturity (in years)
        sigma (float): implied volatility of FX rate

    Returns:
        value (float): value of option 
        delta (float): first derivative of value w.r.t. spot rate
        gamma (float): second derivative of value w.r.t. spot rate
        theta (float): first derivative of value w.r.t. maturity
    """
    d_1 = (np.log(S / K) + (r_d - r_f + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d_2 = d_1 - sigma * np.sqrt(T)
    
    delta = np.exp(- r_f * T) * norm.cdf(d_1)
    gamma = np.exp(- r_f * T) * norm.pdf(d_1) / (S * sigma * np.sqrt(T))
    theta = - S * norm.pdf(d_1) * sigma * np.exp(- r_f * T) / (2 * np.sqrt(T)) + \
        r_f * S * norm.cdf(d_1) * np.exp(- r_f * T) - \
        r_d * K * np.exp(- r_d * T) * norm.cdf(d_2)
    
    value = S * np.exp(-r_f * T) * norm.cdf(d_1) - K * np.exp(- r_d * T) * norm.cdf(d_2)
    
    return value, delta, gamma, theta

In [87]:
value_1, delta_1, gamma_1, theta_1 = garman_kohlhagen(S, K_1, r_d, r_f, T, sigma)

print("First option type")

print(f"Value: {value_1}")
print(f"Delta: {delta_1}")
print(f"Gamma: {gamma_1}")
print(f"Theta: {theta_1}")

First option type
Value: 0.13543575757134307
Delta: 0.9084526019284745
Gamma: 1.3196437265718202
Theta: -0.023167807311047735


In [88]:
value_2, delta_2, gamma_2, theta_2 = garman_kohlhagen(S, K_2, r_d, r_f, T, sigma)

print("Second option type")

print(f"Value: {value_2}")
print(f"Delta: {delta_2}")
print(f"Gamma: {gamma_2}")
print(f"Theta: {theta_2}")

Second option type
Value: 0.011864504480698451
Delta: 0.21346321028058873
Gamma: 2.8446938846556655
Theta: -0.01932787456244206


In [89]:
deltas = np.array([delta_1, -delta_2, 1])
gammas = np.array([gamma_1, -gamma_2, 0])

A = np.array([deltas, gammas, np.ones(3)])
b = np.array([0, 0, 1])

proportions = np.linalg.solve(A, b) # Calculate optimal proportions as solution to system of linear equations

In [90]:
n_1, n_2, n_3 = proportions

print("Optimal proportions")

print(f"First option: {n_1}")
print(f"Second option: {n_2}")
print(f"Underlying asset: {n_3}")

Optimal proportions
First option: 1.5279566829792517
Second option: 0.7088138593904019
Underlying asset: -1.2367705423696536


In [91]:
capital = 100_000

N_2 = capital / (S * abs(n_3) / n_2 - value_2 + value_1 * n_1 / n_2)
N_1 = n_1 * N_2 / n_2
N_3 = abs(n_3) * N_2 / n_2

assert N_1 * value_1 - N_2 * value_2 + N_3 * S == capital # Sanity check

print("Capital allocation")
print(f"First option: {N_1}")
print(f"Second option: {N_2}")
print(f"Underlying asset: {N_3}")

Capital allocation
First option: 95546.68525143311
Second option: 44323.779253333676
Underlying asset: 77338.13861112065


In [92]:
print("Greeks for portfolio")

print(f"Delta: {N_1 * delta_1 - N_2 * delta_2 - N_3}")
print(f"Gamma: {N_1 * gamma_1 - N_2 * gamma_2}")
print(f"Theta: {N_1 * theta_1 - N_2 * theta_2}")

Greeks for portfolio
Delta: -1.4551915228366852e-11
Gamma: -1.4551915228366852e-11
Theta: -1356.922747572724


In [ ]:
print("Value decay over time")

print(f"Value decay per calendar day: {(N_1 * theta_1 - N_2 * theta_2) / 365}")
print(f"Value decay per trading day: {(N_1 * theta_1 - N_2 * theta_2) / 252}")